In [ ]:
from ZV_functions import *
import tensorflow as tf
from matplotlib import pyplot as plt
import matplotlib
import pandas as pd
matplotlib.rcParams.update({'font.size': 16})

In [ ]:
#base_dir='/eos/home-a/ahakimi/www/ZV_analysis'
base_dir='D:\\Travail\\ZV_analysis\\'
year='2018_SR_new'
cut='Boosted_SR'
vers=''


if cut == 'Boosted_SR':
    features =['pt1', 'pt2', 'eta1',
           'eta2', 'mll',
           'FatJet_pt', 'FatJeteta',
           'Zlep_1', 'Zlep_2', 
           'vbs_jet_pt1', 'vbs_jet_pt2',
           'vbs_jet_eta1', 'vbs_jet_eta2',
           'mjj_max', 'detajj_mjjmax']
elif cut == 'Resolved_SR':
    features=['pt1', 'pt2', 'eta1',
           'eta2', 'mll',
           'Zlep_1', 'Zlep_2', 
           'vbs_jet_pt1', 'vbs_jet_pt2',
           'vbs_jet_eta1', 'vbs_jet_eta2',
           'V_jet_pt1', 'V_jet_pt2',
           'V_jet_eta1', 'V_jet_eta2',
           'mjj_max', 'detajj_mjjmax',
           'V_jet_mass']


In [ ]:
X,y, variables_list=prep_data(year, cut, vers, base_dir) #

xlims=[(0,200),(0,150),(-3,3),(-3,3),(0,150),(0,5),(0,10),(0,900),(-5,5),(0,200),(0,1),(-1.5,1.5),(-1.5,1.5),(-1,1),
       (0,200),(0,200),(-5,5),(-5,5),
           (0,200),(0,100),(-5,5),(-5,5),(0,3000),(0,10),(0,4),(0,150)]

#plot_distrib(variables_list, xlims,base_dir, year, cut)
print('NUmber of events: {}, signal : {}, bkg: {}'.format(len(y),len(y[y['signal']==1]),len(y[y['signal']==0])))
print('SOW: {}, signal : {}, bkg: {}'.format(sum(y['weight_']),sum(y['weight_'][y['signal']==1]),sum(y['weight_'][y['signal']==0])))

In [ ]:
data_dir=base_dir+year+'\\'+cut+'\\data'+vers

res_dir=base_dir+year+'\\'+cut+'\\BDT_res'+vers

input_list= features
inputs= [cut+'_'+i for i in input_list]

if not os.path.exists(res_dir):
        os.makedirs(res_dir)

X= pd.DataFrame(np.load(data_dir+'\\X_{}_{}{}.npy'.format(year,cut,vers), allow_pickle=True), columns=variables_list)
X=X[inputs]
y= pd.DataFrame(np.load(data_dir+'\\y_{}_{}{}.npy'.format(year,cut,vers), allow_pickle=True), columns= ['signal','sample','group','weight_', 'w'])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.2, random_state=42)


dtrain=xgb.DMatrix(data=X_train,label=y_train['signal'], feature_names=features_Resolved, weight=y_train['w'])
dtest=xgb.DMatrix(data=X_test,label=y_test['signal'], feature_names=features_Resolved, weight=y_test['w'])
param = {
    # Parameters that we are going to tune.
    'max_depth':4,
    'min_child_weight': sum(y['w'])/len(y),
    'eta':0.05,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    # Other parameters
    'objective':'binary:logistic',
    'eval_metric': 'auc', 
}

#use gpu if available
gpu=tf.test.is_gpu_available(
    cuda_only=False, min_cuda_compute_capability=None
)
if gpu==True:
    param['tree_method']='gpu_hist'
    

num_boost_round=400
es_rounds=10
#watchlist=
progress={}

metrics={'logloss','auc'} #last one used for es




# grid search:
gridsearch_params = [
    (max_depth, eta, l1, l2)
    for max_depth in [4,5,6,8,10]
    for eta in [0.01,0.05,0.1,0.2,0.3]
    for l1 in [0.01, 0.1,1,10] #alpha, def= 0
    for l2 in [0.01, 0.1,1,10] #lambda, def=1
]
tot_iter=len(gridsearch_params)
n_iter=0
max_score = 0
best_params = None
for max_depth, eta, l1, l2 in gridsearch_params:
    n_iter+=1
    print("CV {}/{} with max_depth={}, eta={}, l1={}, l2={}".format(n_iter, tot_iter,
                             max_depth, eta, l1, l2))
    # Update our parameters
    param['max_depth'] = max_depth
    param['eta'] = eta
    param['alpha']= l1
    param['lambda']=l2
    # Run CV
    cv_results = xgb.cv(
        param,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'auc'},
        early_stopping_rounds=es_rounds
    )
    # Update best MAE
    alpha=1
    mean_score = (cv_results['test-auc-mean']-alpha*abs(cv_results['test-auc-mean']-cv_results['train-auc-mean'])).max()
    boost_rounds = (cv_results['test-auc-mean']-alpha*abs(cv_results['test-auc-mean']-cv_results['train-auc-mean'])).argmax()
    print("\tScore {} in {} rounds".format(mean_score, boost_rounds))
    if mean_score > max_score:
        max_score = mean_score
        best_params = (max_depth,eta, l1, l2)
print("Best params: {},  Score: {}".format(best_params, max_score))